
# Setting Up Notebook

Note :

 - *This notebook requires GPU runtime.* <br>
 - *No files need to be added in notebook. <br>Data is being implicitly downloaded in the notebook.* <br>.

In [ ]:
!nvidia-smi

Thu Mar 25 08:09:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -U git+https://github.com/rwightman/pytorch-image-models.git --quiet
!pip install -U albumentations --quiet
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.nn.modules.loss import _WeightedLoss
from torch.nn.modules import Linear
import torch.nn.functional as F
from sklearn.metrics import f1_score,balanced_accuracy_score,accuracy_score,precision_score,recall_score

import timm

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
from scipy.ndimage.interpolation import zoom
! pip install --upgrade albumentations
import albumentations as A

     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 952kB 29.9MB/s 
     |████████████████████████████████| 37.6MB 80kB/s 


In [ ]:
CFG = {
    'fold_num': 5,
    'seed': 42,
    'model_arch': 'resnet18',
    'img_size': 64,
    'epochs': 30,
    'train_bs': 64,
    'valid_bs': 64,
    'T_0': 30,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 2,
    'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0'
}

device = torch.device(CFG['device'])

# Helper Functions

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

seed_everything(CFG['seed'])

# Dataset

Helper Class for Dataloader

In [ ]:
class BoschDataset(Dataset):
    def __init__(self, df, 
                 transforms=None, 
                 output_label=True, 
                 one_hot_label=False,
                ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.output_label = output_label
        self.one_hot_label = one_hot_label
        
        if output_label == True:
            self.labels = self.df['label'].values
            #print(self.labels)
            
            if one_hot_label is True:
                self.labels = np.eye(self.df['label'].max()+1)[self.labels]
                #print(self.labels)
            
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        # get labels
        if self.output_label:
            target = self.labels[index]
        img  = get_img("{}".format(self.df.loc[index]['image']))

        if self.transforms:
            img = self.transforms(image=img)['image']
        if self.output_label == True:
            return img, target
        else:
            return img

# Preprocessing 
Defining Train\Validation Image Augmentations

In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
            Resize(CFG['img_size'], CFG['img_size'],p=1.0,always_apply=True),
            # Transpose(p=0.5),
            # HorizontalFlip(p=0.5),
            # VerticalFlip(p=0.5),
            # ShiftScaleRotate(p=0.5),
            # HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            # RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            # CoarseDropout(p=0.5),
            # Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms():
    return Compose([
            # CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            Resize(CFG['img_size'], CFG['img_size'],p=1.0,always_apply=True),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

# Training APIs
Helper function for training the model

In [ ]:
def prepare_dataloader(train_df,valid_df):
    
    # from catalyst.data.sampler import BalanceClassSampler
    
    train_ds = BoschDataset(train_df, transforms=get_train_transforms(), output_label=True, one_hot_label=False)
    valid_ds = BoschDataset(valid_df, transforms=get_valid_transforms(), output_label=True)
    
    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CFG['train_bs'],
        pin_memory=False,
        drop_last=False,
        shuffle=True,        
        num_workers=CFG['num_workers'],
        #sampler=BalanceClassSampler(labels=train_['label'].values, mode="downsampling")
    )
    val_loader = torch.utils.data.DataLoader(
        valid_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=False,
    )
    return train_loader, val_loader

def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device, scaler, scheduler=None, schd_batch_update=False):
    model.train()

    t = time.time()
    running_loss = None

    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()

        #print(image_labels.shape, exam_label.shape)
        with autocast():
            image_preds = model(imgs)   #output = model(input)
            #print(image_preds.shape, exam_pred.shape)

            loss = loss_fn(image_preds, image_labels)
            
            scaler.scale(loss).backward()

            if running_loss is None:
                running_loss = loss.item()
            else:
                running_loss = running_loss * .99 + loss.item() * .01

            if ((step + 1) %  CFG['accum_iter'] == 0) or ((step + 1) == len(train_loader)):
                # may unscale_ here if desired (e.g., to allow clipping unscaled gradients)

                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad() 
                
                if scheduler is not None and schd_batch_update:
                    scheduler.step()

            if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(train_loader)):
                description = f'epoch {epoch} loss: {running_loss:.4f}'
                
                pbar.set_description(description)
                
    if scheduler is not None and not schd_batch_update:
        scheduler.step()
    return running_loss
        
def valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False):
    model.eval()

    t = time.time()
    loss_sum = 0
    sample_num = 0
    image_preds_all = []
    image_targets_all = []
    
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        image_preds = model(imgs)   #output = model(input)
        #print(image_preds.shape, exam_pred.shape)
        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]
        
        loss = loss_fn(image_preds, image_labels)
        
        loss_sum += loss.item()*image_labels.shape[0]
        sample_num += image_labels.shape[0]  

        if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(val_loader)):
            description = f'epoch {epoch} loss: {loss_sum/sample_num:.4f}'
            pbar.set_description(description)
    
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    weighted_f1_score=f1_score(image_targets_all,image_preds_all,average='weighted')
    macro_f1_score=f1_score(image_targets_all,image_preds_all,average='macro')
    balanced_accuracy_score1=balanced_accuracy_score(image_targets_all,image_preds_all)
    accuracy_score1=accuracy_score(image_targets_all,image_preds_all)
    precision_score1=precision_score(image_targets_all,image_preds_all,average='macro')
    recall_score1=recall_score(image_targets_all,image_preds_all,average='macro')
    print('F1 Score weighted ={:.4f} F1 Score macro ={:.4f} Balanced Accuracy score = {:.4f} Accuracy score ={:.4f} Precision score ={:.4f} Recall score ={:.4f}'
    .format(weighted_f1_score,macro_f1_score,balanced_accuracy_score1,accuracy_score1,precision_score1,recall_score1))
    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()
    return loss_sum/sample_num,weighted_f1_score,macro_f1_score,balanced_accuracy_score1,accuracy_score1,precision_score1,recall_score1


def test_df(model, val_df, loss_fn):

  train_loader, val_loader = prepare_dataloader(val_df, val_df)
  device = torch.device(CFG['device'])
  model.to(device)

  with torch.no_grad():
      valid_one_epoch(0, model, loss_fn, val_loader, device, 
                                    scheduler=None, schd_loss_update=False)

# Data and Drive

Downloading dataset and Mounting Google Drive for saving checkpoints.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if os.path.exists('/content/drive/MyDrive/BoschTrain/{}'.format(CFG['model_arch']))==False:
  os.makedirs('/content/drive/MyDrive/BoschTrain/{}'.format(CFG['model_arch']))

In [ ]:
!gdown --id 1swc4pO3Qe48HDsjrzwp77cqrgPmiPZHI
!unzip -qq Bosch_48classAug.zip -d 'GTSRB_Challenge'
!rm Bosch_48classAug.zip

Downloading...
From: https://drive.google.com/uc?id=1swc4pO3Qe48HDsjrzwp77cqrgPmiPZHI
To: /content/Bosch_48classAug.zip
642MB [00:07, 90.1MB/s]


# Preparing Dataset

*   v1 : trainv1, testv1 - Original Dataset of 43 Classes
*   v2 : trainv2, testv2 - Original Dataset of 43 Classes + 5 new classes
*   v3 : trainv3, testv3  - Original Dataset of 43 Classes + 5 new classes + added augmentations generated dataset creation UI





In [ ]:
trainv3 = pd.read_csv('/content/GTSRB_Challenge/train.csv')
testv3 = pd.read_csv('/content/GTSRB_Challenge/test.csv')
testv3['image'] = 'GTSRB_Challenge/test/' + testv3['image'].astype(str)

trainv2 = trainv3[trainv3['image'].str.contains('GTSRB_Challenge/train')].copy().reset_index(drop=True)
testv2 = testv3[~testv3['image'].str.contains('Aug')].copy().reset_index(drop=True)

trainv1 = trainv2.loc[trainv2['label'] < 43 ].copy().reset_index(drop=True)
testv1 = testv2.loc[testv2['label'] < 43 ].copy().reset_index(drop=True)

# Main Train Function

This function trains given model on the input training and validation data.<br>
Checkpoints are handled here.

In [ ]:
def train(train_df, val_df, model, weights = False):

    train_loader, val_loader = prepare_dataloader(train_df, val_df)

    list_train_loss,list_val_loss,f1_weighted_list,f1_macro_list,balanced_accuracy_list,accuracy_list,precision_list,recall_list=[],[],[],[],[],[],[],[]
    scaler = GradScaler()   
    optimizer = torch.optim.Adam(model.parameters(), lr=CFG['lr'], weight_decay=CFG['weight_decay'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=CFG['T_0'], T_mult=1, eta_min=CFG['min_lr'], last_epoch=-1)

    if weights:

      class_counts = train_df.groupby('label').count().image.values
      class_weights = sum(class_counts)/class_counts
      class_weights = class_weights/max(class_weights)
      loss_tr = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weights)).to(device)

      class_counts = val_df.groupby('label').count().image.values
      class_weights = sum(class_counts)/class_counts
      class_weights = class_weights/max(class_weights)
      loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weights)).to(device)

    else:
      loss_tr = nn.CrossEntropyLoss().to(device) 
      loss_fn = nn.CrossEntropyLoss().to(device)

    best_f1_macro = 0

    for epoch in range(CFG['epochs']):
        train_loss=train_one_epoch(epoch, model, loss_tr, optimizer, train_loader, device, scaler=scaler, scheduler=scheduler, schd_batch_update=False)

        with torch.no_grad():
            loss_sum,weighted_f1_score,macro_f1_score,balanced_accuracy_score1,accuracy_score1,precision_score1,recall_score1=valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False)

        if macro_f1_score > best_f1_macro:
            torch.save({'model_state_dict':model.state_dict(),'optimizer_state_dict':optimizer.state_dict(),'scheduler_state_dict':scheduler.state_dict(),'epoch':epoch,'train_loss':train_loss,'validation_loss':loss_sum},'/content/drive/MyDrive/BoschTrain/{}/{}{}best.pth'.format(CFG['model_arch'],train_df.label.nunique(),int(weights)))
            best_f1_macro = macro_f1_score
            print("Best f1_macro_score updated : {}".format(best_f1_macro))

        with open("/content/drive/MyDrive/BoschTrain/{}/log{}{}.txt".format(CFG['model_arch'],train_df.label.nunique(),int(weights)),'a') as file:
          file.write(f'Epoch={epoch} Train_loss={train_loss} Validation_loss={loss_sum} F1 score weighted={weighted_f1_score} F1 Score macro={macro_f1_score} Balanced Accuracy score = {balanced_accuracy_score1} Accuracy score ={accuracy_score1} Precision score ={precision_score1} Recall score ={recall_score1}\n\n')

        list_train_loss.append(train_loss);list_val_loss.append(loss_sum);f1_weighted_list.append(weighted_f1_score);
        f1_macro_list.append(macro_f1_score);balanced_accuracy_list.append(balanced_accuracy_score1);accuracy_list.append(accuracy_score1);
        recall_list.append(recall_score1);precision_list.append(precision_score1);

    torch.save({'train_loss':list_train_loss,'val_loss':list_val_loss,'f1_weighted_score':f1_weighted_list,'f1_macro_score':macro_f1_score,
                'balanced_accuracy':balanced_accuracy_list,'accuracy':accuracy_list,'precision':precision_list,'recall':recall_list},
               '/content/drive/MyDrive/BoschTrain/{}/{}{}list.pth'.format(CFG['model_arch'],train_df.label.nunique(),int(weights)))
    
    torch.save({'model_state_dict':model.state_dict(),'optimizer_state_dict':optimizer.state_dict(),'scheduler_state_dict':scheduler.state_dict(),
                'epoch':epoch,'train_loss':train_loss,'validation_loss':loss_sum},
               '/content/drive/MyDrive/BoschTrain/{}/{}{}last.pth'.format(CFG['model_arch'],train_df.label.nunique(),int(weights)))

    del optimizer, train_loader, val_loader, scaler, scheduler
    torch.cuda.empty_cache()

    return model,loss_fn

# Training on V1

In [ ]:
train_df = trainv1
val_df  = testv1
model_wt_path = ''

model = timm.create_model(CFG['model_arch'],pretrained=True,num_classes=train_df.label.nunique())
model.to(device)

if model_wt_path:
  model.load_state_dict(torch.load(model_wt_path)['model_state_dict'])

model,loss_fn = train(train_df, val_df, model)


F1 Score weighted =0.9534 F1 Score macro =0.9158 Balanced Accuracy score = 0.9201 Accuracy score =0.9527 Precision score =0.9177 Recall score =0.9201
Best f1_macro_score updated : 0.9158308148479184



F1 Score weighted =0.9597 F1 Score macro =0.9311 Balanced Accuracy score = 0.9293 Accuracy score =0.9599 Precision score =0.9392 Recall score =0.9293
Best f1_macro_score updated : 0.9310750128173991



F1 Score weighted =0.9605 F1 Score macro =0.9351 Balanced Accuracy score = 0.9374 Accuracy score =0.9607 Precision score =0.9370 Recall score =0.9374
Best f1_macro_score updated : 0.9351180279470622



F1 Score weighted =0.9616 F1 Score macro =0.9377 Balanced Accuracy score = 0.9413 Accuracy score =0.9616 Precision score =0.9399 Recall score =0.9413
Best f1_macro_score updated : 0.9376701640085768



F1 Score weighted =0.9606 F1 Score macro =0.9388 Balanced Accuracy score = 0.9431 Accuracy score =0.9603 Precision score =0.9422 Recall score =0.9431
Best f1_macro_score updated : 0.9387918627701702


...


F1 Score weighted =0.9868 F1 Score macro =0.9814 Balanced Accuracy score = 0.9810 Accuracy score =0.9868 Precision score =0.9824 Recall score =0.9810
Best f1_macro_score updated : 0.9814490943640312



F1 Score weighted =0.9872 F1 Score macro =0.9816 Balanced Accuracy score = 0.9817 Accuracy score =0.9872 Precision score =0.9821 Recall score =0.9817
Best f1_macro_score updated : 0.9816496723929072



F1 Score weighted =0.9866 F1 Score macro =0.9810 Balanced Accuracy score = 0.9794 Accuracy score =0.9867 Precision score =0.9831 Recall score =0.9794



F1 Score weighted =0.9864 F1 Score macro =0.9803 Balanced Accuracy score = 0.9802 Accuracy score =0.9864 Precision score =0.9808 Recall score =0.9802



F1 Score weighted =0.9856 F1 Score macro =0.9790 Balanced Accuracy score = 0.9778 Accuracy score =0.9857 Precision score =0.9809 Recall score =0.9778


### testing on v1 + added augmentations generated dataset creation UI

In [ ]:
test_df(model, testv3.loc[testv3['label'] < 43 ].copy().reset_index(drop=True) , loss_fn)


F1 Score weighted =0.9690 F1 Score macro =0.9610 Balanced Accuracy score = 0.9554 Accuracy score =0.9690 Precision score =0.9685 Recall score =0.9554
